In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from pyvi import ViTokenizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import string

### Data

In [2]:
class Data:
    
    def load_raw_data(self, filename, is_train=True):
        ID = []
        label = []
        sentence = []
        review = ''        
        if is_train:
            with open(filename, 'r', encoding='utf-8') as file:
                for line in file:
                    if line.startswith('train_'):
                        ID.append(line.replace('\n',''))
                    elif line[:-1].isdigit():
                        review = review.replace('\n',' ')
                        review = ' '.join((review.lower()).split())
                        sentence.append(review)
                        review = ''
                        label.append(int(line[:-1]))
                    else:
                        review = review +' '+ line
        else:
            with open(filename, 'r', encoding='utf-8') as file:
                for line in file:
                    if line.startswith('test_'):
                        review = review.replace('\n',' ')
                        review = ' '.join((review.lower()).split())
                        sentence.append(review)
                        review = ''
                        ID.append(line.replace('\n',''))
                    else:
                        review = review +' '+ line
            review = review.replace('\n',' ')
            review = re.sub('['+string.punctuation+']', ' ', review)
            review = ' '.join((review.lower()).split())
            sentence.append(review)
            del sentence[0]
        return (ID, label, sentence)
    
    def pre_processing(self, text):
        
        # Xoa icon linh tinh
        emoji_pattern = re.compile("["
                       u"\U0001F600-\U0001F64F"  # emoticons
                       u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                       u"\U0001F680-\U0001F6FF"  # transport & map symbols
                       u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                       u"\U00002702-\U000027B0"
                       u"\U000024C2-\U0001F251"
                       u"\U0001f926-\U0001f937"
                       u"\u200d"
                       u"\u2640-\u2642" 
                       "]+", flags=re.UNICODE)
        text = emoji_pattern.sub(r'', text)
        
        # Chuan hoa tieng viet, tieng anh, thuat ngu
        replace_list={
            'òa': 'oà', 'óa': 'oá', 'ỏa': 'oả', 'õa': 'oã', 'ọa': 'oạ', 'òe': 'oè', 'óe': 'oé','ỏe': 'oẻ',
        'õe': 'oẽ', 'ọe': 'oẹ', 'ùy': 'uỳ', 'úy': 'uý', 'ủy': 'uỷ', 'ũy': 'uỹ','ụy': 'uỵ', 'uả': 'ủa',
        'ả': 'ả', 'ố': 'ố', 'u´': 'ố','ỗ': 'ỗ', 'ồ': 'ồ', 'ổ': 'ổ', 'ấ': 'ấ', 'ẫ': 'ẫ', 'ẩ': 'ẩ',
        'ầ': 'ầ', 'ỏ': 'ỏ', 'ề': 'ề','ễ': 'ễ', 'ắ': 'ắ', 'ủ': 'ủ', 'ế': 'ế', 'ở': 'ở', 'ỉ': 'ỉ',
        'ẻ': 'ẻ', 'àk': u' à ','aˋ': 'à', 'iˋ': 'ì', 'ă´': 'ắ','ử': 'ử', 'e˜': 'ẽ', 'y˜': 'ỹ', 'a´': 'á',
        #Chuẩn hóa 1 số sentiment words/English words
        ':))': '  positive ', ':)': ' positive ', 'ô kêi': ' ok ', 'okie': ' ok ', ' o kê ': ' ok ',
        'okey': ' ok ', 'ôkê': ' ok ', ' oki ': ' ok ', ' oke ':  ' ok ',' okay':' ok ','okê':' ok ',
        ' tks ': u' cám ơn ', 'thks': u' cám ơn ', 'thanks': u' cám ơn ', 'ths': u' cám ơn ', 'thank': u' cám ơn ', 'cam on':u'cám ơn',u'cảm ơn':'cám ơn',
        ' not ': u' không ',' khoong ': ' không ', u' kg ': u' không ','ko ': 'không', ' k ': u' không ',' kh ':u' không ',' kô ':u' không ','hok':u' không ',' kp ': u' không phải ',u' kô ': u' không ', ' ko ': u' không ', u' ko ': u' không ', u' k ': u' không ', 'khong': u' không ', u' hok ': u' không ',' k ':u' không ',u'chẳng':u'không',u'đéo':u'không',
        'he he': ' positive ','hehe': ' positive ','hihi': ' positive ', 'haha': ' positive ', 'hjhj': ' positive ',
        ' lol ': ' negative ',' cc ': ' negative ','cute': u' dễ thương ','huhu': ' negative ', ' vs ': u' với ', 'wa': ' quá ', 'wá': u' quá', ' j ': u' gì ', '“': ' ',
        ' sz ': u' cỡ ', 'size': u' cỡ ', u' đx ': u' được ', 'dk': u' được ', 'dc': u' được ', 'đk': u' được ',
        'đc': u' được ','authentic': u' chuẩn chính hãng ',u' aut ': u' chuẩn chính hãng ', u' auth ': u' chuẩn chính hãng ', 'thick': u' positive ', 'store': u' cửa hàng ',
        'shop': u' cửa hàng ', 'sp': u' sản phẩm ', 'gud': u' tốt ','god': u' tốt ','wel done':' tốt ', 'good': u' tốt ', 'gút': u' tốt ',
        'sấu': u' xấu ','gut': u' tốt ', u' tot ': u' tốt ', u' nice ': u' tốt ', 'perfect': 'rất tốt', 'bt': u' bình thường ',
        'time': u' thời gian ', 'qá': u' quá ', u' ship ': u' giao hàng ', u' m ': u' mình ', u' mik ': u' mình ',
        'ể': 'ể', 'product': 'sản phẩm', 'quality': 'chất lượng','chat':' chất ', 'excelent': 'hoàn hảo', 'bad': 'tệ','fresh': ' tươi ','sad': ' tệ ',
        'date': u' hạn sử dụng ', 'hsd': u' hạn sử dụng ','quickly': u' nhanh ', 'quick': u' nhanh ','fast': u' nhanh ','delivery': u' giao hàng ',u' síp ': u' giao hàng ',
        'beautiful': u' đẹp tuyệt vời ', u' tl ': u' trả lời ', u' r ': u' rồi ', u' shopE ': u' cửa hàng ',u' order ': u' đặt hàng ',
        'chất lg': u' chất lượng ',u' sd ': u' sử dụng ',u' dt ': u' điện thoại ',u' nt ': u' nhắn tin ',u' tl ': u' trả lời ',u' sài ': u' xài ',u'bjo':u' bao giờ ',
        'thik': u' thích ',u' sop ': u' cửa hàng ', ' fb ': ' facebook ', ' face ': ' facebook ', ' very ': u' rất ', ' quả ng ': ' quảng ',
        'dep': u' đẹp ',u'xầu':u'xấu',u' xau ': u' xấu ','delicious': u' ngon ', u'hàg': u' hàng ', u' qủa': u' quả', 
        ' iu ': u' yêu ','fake': u' giả mạo ', 'trl': 'trả lời', '><': u' positive ', 
        ' por ': u' tệ ',' poor ': u' tệ ', 'ib':u' nhắn tin ', 'rep':u' trả lời ',u'fback':' feedback ','fedback':' feedback ',
        'qc':u'quảng cáo',u' éo ':' negative ',' bik ':' biết ',
        #dưới 3* quy về 1*, trên 3* quy về 5*
        '6 sao': ' 5star ','6 star': ' 5star ', '5star': ' 5star ','5 sao': ' 5star ','5sao': ' 5star ', '5 sao': '5star',
        'starstarstarstarstar': ' 5star ', '1 sao': ' 1star ', '1sao': ' 1star ','2 sao':' 1star ','2sao':' 1star ', '3 sao':'5star','4 sao':'5star','1 sao':'1star',
        '2 starstar':' 1star ','1star': ' 1star ', '0 sao': ' 1star ', '0star': ' 1star ','1*':' 1star ', '2*':' 1star ','3*':' 5star ','4*':' 5star ','5*':' 5star '
        }
        for k, v in replace_list.items():
            text = text.replace(k, v)
        
        #Xóa dấu câu linh tinh
        text = re.sub('['+string.punctuation+']', ' ', text)
        
        # Tokenizer
        text = ViTokenizer.tokenize(text)
        
        _not = ['không', 'không thể' , 'chẳng', 'đéo', 'đếch', 'kém']
        positive = ['5star', 'positive', 'ưng', 'hài lòng', 'thích',  'ngon', 'tốt']
        negative = ['1start', 'negative', 'lừa','thất vọng', 'chán', 'hợp', 'xấu', 'lừa đảo', 'rất kém','tệ']
        txt_split = [txt.replace('_',' ') for txt in text.split()]
        for i in range(len(txt_split)):
            if txt_split[i] in _not:
                if (i < len(txt_split)-1):
                    if txt_split[i+1] in positive:
                        text = text.replace(txt_split[i]+' '+txt_split[i+1],'negative')
                        txt_split[i]=txt_split[i+1]=''
                    elif txt_split[i+1] in negative:
                        text = text.replace(txt_split[i]+' '+txt_split[i+1],'positive')
                        txt_split[i]=txt_split[i+1]=''
            else:
                if txt_split[i] in negative:
                    text = text + ' negative'
                elif txt_split[i] in positive:
                    text = text + ' positive'
        return text
    
    
    def load_data(self, filename, is_train=True):
        new_sentence = []
        ID, Label, Sentence = self.load_raw_data(filename, is_train)
        for i in range(len(Sentence)):
            new_sentence.append(self.pre_processing(Sentence[i]))
        return ID, Label, Sentence, new_sentence
    
    @staticmethod
    def export_to_excel(data_frame,filename_save, sheet_name='Sheet1'):
        filename_save = str(filename_save)+'.xlsx'
        excel_save = pd.ExcelWriter(filename_save, engine='xlsxwriter')
        data_frame.to_excel(excel_save, sheet_name=sheet_name)
        excel_save.save()

### Multinomial Naive Bayes Model

In [5]:
from sklearn.naive_bayes import MultinomialNB

In [6]:
class NaiveBayesModel:
    def __init__(self):
        self.clf = self._init_pipeline()
    
    @staticmethod
    def _init_pipeline():
        stopwords=('rằng', 'thì', 'mà', 'là', 'thế', 'à', 'ừ', 'vậy', 'như')
        pipe_line = Pipeline([
            ("vectorizer", CountVectorizer(stop_words=stopwords)), # bag of words
            ("tfidf", TfidfTransformer()), # tf-idf
            ("clf", MultinomialNB()) # navie bayes models
        ])
        return pipe_line

### Support Vector Machine - kernel = 'linear'

In [7]:
from sklearn.svm import SVC 

In [8]:
class SVMModel:
    def __init__(self):
        self.clf = self._init_pipeline()
        
    @staticmethod
    def _init_pipeline():
        stopwords=('rằng', 'thì', 'mà', 'là', 'thế', 'à', 'ừ', 'vậy', 'như')
        pipe_line = Pipeline([
            ("vectorizer", CountVectorizer(stop_words=stopwords)), # bag of words
            ("tfidf", TfidfTransformer()), # tf-idf
            ("clf_svm", SVC(kernel='linear', probability=True)) # svm kernel = 'linear'
        ])
        return pipe_line

## Classifier

### Create data frame

In [9]:
ID, Label, Sentence, New_Sentence = Data().load_data(r'D:\Documents\Đồ án 2\Sentiment Analysis\Sentiment data\vibo\train.crash')

In [10]:
train_df = pd.DataFrame([ID, Label, Sentence, New_Sentence]).T

In [11]:
train_df.columns = ['ID', 'Label', 'Sentence', 'New_Sentence']

In [12]:
train_df[:10]

,ID,Label,Sentence,New_Sentence
0,train_000000,0,"""dung dc sp tot cam on shop đóng gói sản phẩm ...",dung được sản_phẩm tốt cám_ơn cửa_hàng đóng_gó...
1,train_000001,0,""" chất lượng sản phẩm tuyệt vời . son mịn nhưn...",chất_lượng sản_phẩm tuyệt_vời son mịn nhưng kh...
2,train_000002,0,""" chất lượng sản phẩm tuyệt vời nhưng k có hộp...",chất_lượng sản_phẩm tuyệt_vời nhưng không có h...
3,train_000003,1,""":(( mình hơi thất vọng 1 chút vì mình đã kỳ v...",mình hơi thất_vọng 1 chút vì mình đã kỳ_vọng c...
4,train_000004,1,"""lần trước mình mua áo gió màu hồng rất ok mà ...",lần trước mình mua áo_gió màu hồng rất ok mà đ...
5,train_000005,0,""" chất lượng sản phẩm tuyệt vời có điều không ...",chất_lượng sản_phẩm tuyệt_vời có_điều không cứ...
6,train_000006,0,"""đã nhận đc hàng rất nhanh mới đặt buổi tối mà...",đã nhận được hàng rất nhanh mới đặt buổi tối m...
7,train_000007,1,"""các siêu phẩm thấy cấu hình toàn tựa tựa nhau...",các siêu phẩm thấy cấu_hình toàn tựa tựa nhau ...
8,train_000008,0,"""hàng ship nhanh chất lượng tốt tư vấn nhiệt t...",hàng giao hàng nhanh chất_lượng tốt tư_vấn nhi...
9,train_000009,1,"""đồng hồ đẹp nhưng 1 cái đứt dây 1 cái k chạy ...",đồng_hồ đẹp nhưng 1 cái đứt dây 1 cái không ch...


In [13]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16087 entries, 0 to 16086
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   ID            16087 non-null  object
 1   Label         16087 non-null  object
 2   Sentence      16087 non-null  object
 3   New_Sentence  16087 non-null  object
dtypes: object(4)
memory usage: 502.8+ KB


### Split train/test data

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
x_train, x_test, y_train, y_test = train_test_split(train_df.loc[:,'New_Sentence'].values, (train_df.loc[:,'Label'].values).astype('int'), test_size = .2, random_state=0)

### Multinomial Naive Bayes

In [16]:
model = NaiveBayesModel()
clf = model.clf.fit(x_train, y_train)
print("Train accuracy:",clf.score(x_train, y_train))
print("Test accuracy:", clf.score(x_test, y_test))

Train accuracy: 0.9108710855544331
Test accuracy: 0.8666873834679926


### SVM kernel = 'linear'

In [17]:
clf_svm = SVMModel().clf.fit(x_train, y_train)
print("Train accuracy:",clf_svm.score(x_train, y_train))
print("Test accuracy:", clf_svm.score(x_test, y_test))

Train accuracy: 0.945139482477271
Test accuracy: 0.8912367930391547


## Input from keyboard

In [18]:
sentiment = ['tích cực', 'tiêu cực']
test_data = {}
test_data['Sentence']= input('Mời bạn nhập bình luận: ')
test_df = pd.DataFrame(test_data,index=[0])
predict = clf_svm.predict(test_df['Sentence'])[0]
print(sentiment[predict])
print(clf_svm.predict_proba(test_df['Sentence']))

Mời bạn nhập bình luận: tuyệt vời đấy nhưng không mua
tích cực
[[0.71058395 0.28941605]]


### Filter error label predict

In [19]:
predict_label = clf_svm.predict(train_df['New_Sentence'])
probability = clf_svm.predict_proba(train_df['New_Sentence'])

In [20]:
train_df['Positive_proba'] = probability[:,0]
train_df['Negative_proba'] = probability[:,1]
train_df['Predict_label'] = predict_label

In [21]:
error_predict_df = train_df[train_df['Label']!= predict_label]

In [22]:
length = []
for i in range(error_predict_df.shape[0]):
    length.append(len((error_predict_df['Sentence'][error_predict_df.index[i]]).split()))
error_predict_df['Length_sentence'] = length

c:\users\manhlk\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [23]:
error_predict_df

,ID,Label,Sentence,New_Sentence,Positive_proba,Negative_proba,Predict_label,Length_sentence
2,train_000002,0,""" chất lượng sản phẩm tuyệt vời nhưng k có hộp...",chất_lượng sản_phẩm tuyệt_vời nhưng không có h...,0.408030,0.591970,1,19
32,train_000032,0,"""sạc tự động ngắt pin""",sạc tự_động ngắt pin,0.149288,0.850712,1,5
50,train_000050,1,""" đóng gói sản phẩm rất kémtư vấn k nhiệt tình""",đóng_gói sản_phẩm rất kémtư vấn không nhiệt_tình,0.825142,0.174858,0,11
52,train_000052,0,"""e xay đá mà bị gãy ko xay dc ạ""",e xay đá mà bị gãy khôngxay được ạ,0.195948,0.804052,1,10
57,train_000057,0,"""sản phẩm k như mình mong đợi..k cài dc đồ chơ...",sản_phẩm không như mình mong_đợi k cài được đồ...,0.262330,0.737670,1,18
...,...,...,...,...,...,...,...,...
15975,train_015975,1,"""mình đặt 5 gói mặt nạ su:m trắng mà shop giao...",mình đặt 5 gói mặt_nạ su m trắng mà cửa_hàng g...,0.601992,0.398008,0,15
15976,train_015976,0,"""""hồi đó mình sài s5, nhưng chạy theo phong tr...",hồi đó mình xài s5 nhưng chạy theo phong_trào ...,0.282036,0.717964,1,99
15993,train_015993,0,"""như l""",như l,0.438406,0.561594,1,2
16029,train_016029,0,"""""rất ấn tượng.""""",rất ấn_tượng,0.456237,0.543763,1,3


### Export to excel

In [ ]:
Data.export_to_excel(error_predict_df,'error_data','False predict')

### Relabel

In [18]:
relabel_error_df = pd.read_excel(r'C:\Users\MANHLK\Documents\Machine_Learning\Sentiment\error_predict.xlsx')
relabel_error_df.head()

,Unnamed: 0,ID,Label,Sentence,New_Sentence,Positive_proba,Negative_proba,Predict_label,Length_sentence,ReLabel
0,2,train_000002,0,""" chất lượng sản phẩm tuyệt vời nhưng k có hộp...",chất_lượng sản_phẩm tuyệt_vời nhưng không có h...,0.405822,0.594178,1,19,2
1,32,train_000032,0,"""sạc tự động ngắt pin""",sạc tự_động ngắt pin,0.235148,0.764852,1,5,1
2,50,train_000050,1,""" đóng gói sản phẩm rất kémtư vấn k nhiệt tình""",đóng_gói sản_phẩm rất kémtư vấn không nhiệt_tình,0.832132,0.167868,0,11,1
3,52,train_000052,0,"""e xay đá mà bị gãy ko xay dc ạ""",e xay đá mà bị gãy không xay được ạ,0.058185,0.941815,1,10,1
4,57,train_000057,0,"""sản phẩm k như mình mong đợi..k cài dc đồ chơ...",sản_phẩm không như mình mong_đợi k cài được đồ...,0.195521,0.804479,1,18,1


In [19]:
count_sentiment = pd.value_counts(relabel_error_df['ReLabel'])
count_sentiment

 1    611
 0    200
 2    188
-1    111
Name: ReLabel, dtype: int64

In [20]:
for i in range(relabel_error_df.shape[0]):
    relabel = relabel_error_df['ReLabel'][i]
    if relabel == 2 or relabel == -1:
        relabel = np.random.randint(0,2)
        train_df['Label'][relabel_error_df.iloc[:,0].values[i]] = relabel
    else:
        train_df['Label'][relabel_error_df.iloc[:,0].values[i]] = relabel

In [21]:
Data.export_to_excel(train_df,'train','Sheet 1')

In [22]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16087 entries, 0 to 16086
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   ID            16087 non-null  object
 1   Label         16087 non-null  object
 2   Sentence      16087 non-null  object
 3   New_Sentence  16087 non-null  object
dtypes: object(4)
memory usage: 502.8+ KB


### Load SVM Model again

In [23]:
x_train, x_test, y_train, y_test = train_test_split(train_df.loc[:,'New_Sentence'].values, (train_df.loc[:,'Label'].values).astype('int'), test_size = .2, random_state=0)

In [24]:
clf_svm = SVMModel().clf.fit(x_train, y_train)
print("Train accuracy:",clf_svm.score(x_train, y_train))
print("Test accuracy:", clf_svm.score(x_test, y_test))

Train accuracy: 0.9759111042039008
Test accuracy: 0.9403356121814792


In [26]:
data_aivivn = train_df[['Label', 'Sentence']]

In [28]:
Data.export_to_excel(data_aivivn,'data_aivivn','Sheet_1')